# Part 1: Basic Snowflake Setup

This section covers the basic setup for the Snowflake AI Demo. It assumes you have already run the setup script to clone the repository and load the data into an internal stage. The rest of this section will cover table creation and data loading.

## 1. Table Creation

Create all the dimension and fact tables needed for the demo. This includes:
- **Dimension Tables**: Reference data like products, customers, vendors, etc.
- **Fact Tables**: Transactional data for sales, finance, marketing, and HR
- **Salesforce CRM Tables**: Integration with Salesforce data

### 1.1 Dimension Tables

Dimension tables contain descriptive attributes and serve as lookup tables.

In [ ]:
USE ROLE SF_INTELLIGENCE_DEMO;
USE DATABASE SF_AI_DEMO;
USE SCHEMA DEMO_SCHEMA;

-- Product Category Dimension
CREATE OR REPLACE TABLE product_category_dim (
    category_key INT PRIMARY KEY,
    category_name VARCHAR(100) NOT NULL,
    vertical VARCHAR(50) NOT NULL
);

-- Product Dimension
CREATE OR REPLACE TABLE product_dim (
    product_key INT PRIMARY KEY,
    product_name VARCHAR(200) NOT NULL,
    category_key INT NOT NULL,
    category_name VARCHAR(100),
    vertical VARCHAR(50)
);

In [ ]:
-- Vendor Dimension
CREATE OR REPLACE TABLE vendor_dim (
    vendor_key INT PRIMARY KEY,
    vendor_name VARCHAR(200) NOT NULL,
    vertical VARCHAR(50) NOT NULL,
    address VARCHAR(200),
    city VARCHAR(100),
    state VARCHAR(10),
    zip VARCHAR(20)
);

-- Customer Dimension
CREATE OR REPLACE TABLE customer_dim (
    customer_key INT PRIMARY KEY,
    customer_name VARCHAR(200) NOT NULL,
    industry VARCHAR(100),
    vertical VARCHAR(50),
    address VARCHAR(200),
    city VARCHAR(100),
    state VARCHAR(10),
    zip VARCHAR(20)
);

-- Account Dimension (Finance)
CREATE OR REPLACE TABLE account_dim (
    account_key INT PRIMARY KEY,
    account_name VARCHAR(100) NOT NULL,
    account_type VARCHAR(50)
);

In [ ]:
-- Department Dimension
CREATE OR REPLACE TABLE department_dim (
    department_key INT PRIMARY KEY,
    department_name VARCHAR(100) NOT NULL
);

-- Region Dimension
CREATE OR REPLACE TABLE region_dim (
    region_key INT PRIMARY KEY,
    region_name VARCHAR(100) NOT NULL
);

-- Sales Rep Dimension
CREATE OR REPLACE TABLE sales_rep_dim (
    sales_rep_key INT PRIMARY KEY,
    rep_name VARCHAR(200) NOT NULL,
    hire_date DATE
);

In [ ]:
-- Campaign Dimension (Marketing)
CREATE OR REPLACE TABLE campaign_dim (
    campaign_key INT PRIMARY KEY,
    campaign_name VARCHAR(300) NOT NULL,
    objective VARCHAR(100)
);

-- Channel Dimension (Marketing)
CREATE OR REPLACE TABLE channel_dim (
    channel_key INT PRIMARY KEY,
    channel_name VARCHAR(100) NOT NULL
);

-- Employee Dimension (HR)
CREATE OR REPLACE TABLE employee_dim (
    employee_key INT PRIMARY KEY,
    employee_name VARCHAR(200) NOT NULL,
    gender VARCHAR(1),
    hire_date DATE
);

-- Job Dimension (HR)
CREATE OR REPLACE TABLE job_dim (
    job_key INT PRIMARY KEY,
    job_title VARCHAR(100) NOT NULL,
    job_level INT
);

-- Location Dimension (HR)
CREATE OR REPLACE TABLE location_dim (
    location_key INT PRIMARY KEY,
    location_name VARCHAR(200) NOT NULL
);

### 1.2 Fact Tables

Fact tables contain the measurable, quantitative data for business processes. These tables store the actual transactions and events.

In [ ]:
-- Sales Fact Table
CREATE OR REPLACE TABLE sales_fact (
    sale_id INT PRIMARY KEY,
    date DATE NOT NULL,
    customer_key INT NOT NULL,
    product_key INT NOT NULL,
    sales_rep_key INT NOT NULL,
    region_key INT NOT NULL,
    vendor_key INT NOT NULL,
    amount DECIMAL(10,2) NOT NULL,
    units INT NOT NULL
);

-- Finance Transactions Fact Table
CREATE OR REPLACE TABLE finance_transactions (
    transaction_id INT PRIMARY KEY,
    date DATE NOT NULL,
    account_key INT NOT NULL,
    department_key INT NOT NULL,
    vendor_key INT NOT NULL,
    product_key INT NOT NULL,
    customer_key INT NOT NULL,
    amount DECIMAL(12,2) NOT NULL
);

In [ ]:
-- Marketing Campaign Fact Table
CREATE OR REPLACE TABLE marketing_campaign_fact (
    campaign_fact_id INT PRIMARY KEY,
    date DATE NOT NULL,
    campaign_key INT NOT NULL,
    product_key INT NOT NULL,
    channel_key INT NOT NULL,
    region_key INT NOT NULL,
    spend DECIMAL(10,2) NOT NULL,
    leads_generated INT NOT NULL,
    impressions INT NOT NULL
);

-- HR Employee Fact Table
CREATE OR REPLACE TABLE hr_employee_fact (
    hr_fact_id INT PRIMARY KEY,
    date DATE NOT NULL,
    employee_key INT NOT NULL,
    department_key INT NOT NULL,
    job_key INT NOT NULL,
    location_key INT NOT NULL,
    salary DECIMAL(10,2) NOT NULL,
    attrition_flag INT NOT NULL
);

### 1.3 Salesforce CRM Tables

In [ ]:
-- Salesforce Accounts Table
CREATE OR REPLACE TABLE sf_accounts (
    account_id VARCHAR(20) PRIMARY KEY,
    account_name VARCHAR(200) NOT NULL,
    customer_key INT NOT NULL,
    industry VARCHAR(100),
    vertical VARCHAR(50),
    billing_street VARCHAR(200),
    billing_city VARCHAR(100),
    billing_state VARCHAR(10),
    billing_postal_code VARCHAR(20),
    account_type VARCHAR(50),
    annual_revenue DECIMAL(15,2),
    employees INT,
    created_date DATE
);

-- Salesforce Opportunities Table
CREATE OR REPLACE TABLE sf_opportunities (
    opportunity_id VARCHAR(20) PRIMARY KEY,
    sale_id INT,
    account_id VARCHAR(20) NOT NULL,
    opportunity_name VARCHAR(200) NOT NULL,
    stage_name VARCHAR(100) NOT NULL,
    amount DECIMAL(15,2) NOT NULL,
    probability DECIMAL(5,2),
    close_date DATE,
    created_date DATE,
    lead_source VARCHAR(100),
    type VARCHAR(100),
    campaign_id INT
);

-- Salesforce Contacts Table
CREATE OR REPLACE TABLE sf_contacts (
    contact_id VARCHAR(20) PRIMARY KEY,
    opportunity_id VARCHAR(20) NOT NULL,
    account_id VARCHAR(20) NOT NULL,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    email VARCHAR(200),
    phone VARCHAR(50),
    title VARCHAR(100),
    department VARCHAR(100),
    lead_source VARCHAR(100),
    campaign_no INT,
    created_date DATE
);

## 2. Data Loading

Load all the demo data from the internal stage into the tables we just created.

### 2.1 Load Dimension Data


In [ ]:
-- Load Product Category Dimension
COPY INTO product_category_dim
FROM @INTERNAL_DATA_STAGE/demo_data/product_category_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Product Dimension
COPY INTO product_dim
FROM @INTERNAL_DATA_STAGE/demo_data/product_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Vendor Dimension
COPY INTO vendor_dim
FROM @INTERNAL_DATA_STAGE/demo_data/vendor_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

In [ ]:
-- Load Customer Dimension
COPY INTO customer_dim
FROM @INTERNAL_DATA_STAGE/demo_data/customer_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Account Dimension
COPY INTO account_dim
FROM @INTERNAL_DATA_STAGE/demo_data/account_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Department Dimension
COPY INTO department_dim
FROM @INTERNAL_DATA_STAGE/demo_data/department_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Region Dimension
COPY INTO region_dim
FROM @INTERNAL_DATA_STAGE/demo_data/region_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

In [ ]:
-- Load Sales Rep Dimension
COPY INTO sales_rep_dim
FROM @INTERNAL_DATA_STAGE/demo_data/sales_rep_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Campaign Dimension
COPY INTO campaign_dim
FROM @INTERNAL_DATA_STAGE/demo_data/campaign_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Channel Dimension
COPY INTO channel_dim
FROM @INTERNAL_DATA_STAGE/demo_data/channel_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Employee Dimension
COPY INTO employee_dim
FROM @INTERNAL_DATA_STAGE/demo_data/employee_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Job Dimension
COPY INTO job_dim
FROM @INTERNAL_DATA_STAGE/demo_data/job_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Location Dimension
COPY INTO location_dim
FROM @INTERNAL_DATA_STAGE/demo_data/location_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

### 2.2 Load Fact Data

Fact tables are loaded after dimensions since they reference dimension keys.

In [ ]:
-- Load Sales Fact
COPY INTO sales_fact
FROM @INTERNAL_DATA_STAGE/demo_data/sales_fact.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Finance Transactions
COPY INTO finance_transactions
FROM @INTERNAL_DATA_STAGE/demo_data/finance_transactions.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Marketing Campaign Fact
COPY INTO marketing_campaign_fact
FROM @INTERNAL_DATA_STAGE/demo_data/marketing_campaign_fact.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load HR Employee Fact
COPY INTO hr_employee_fact
FROM @INTERNAL_DATA_STAGE/demo_data/hr_employee_fact.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

### 2.3 Load Salesforce Data

In [ ]:
-- Load Salesforce Accounts
COPY INTO sf_accounts
FROM @INTERNAL_DATA_STAGE/demo_data/sf_accounts.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Salesforce Opportunities
COPY INTO sf_opportunities
FROM @INTERNAL_DATA_STAGE/demo_data/sf_opportunities.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Salesforce Contacts
COPY INTO sf_contacts
FROM @INTERNAL_DATA_STAGE/demo_data/sf_contacts.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

## 3. Data Verification

Verify that all data has been loaded successfully by checking row counts and showing the created objects.

In [ ]:
-- Show all tables created in the demo schema
SHOW TABLES IN SCHEMA DEMO_SCHEMA;

### 3.1 Verify All Data

In [ ]:
-- Verify data loads with comprehensive row count report
SELECT 'DIMENSION TABLES' as category, '' as table_name, NULL as row_count
UNION ALL
SELECT '', 'product_category_dim', COUNT(*) FROM product_category_dim
UNION ALL
SELECT '', 'product_dim', COUNT(*) FROM product_dim
UNION ALL
SELECT '', 'vendor_dim', COUNT(*) FROM vendor_dim
UNION ALL
SELECT '', 'customer_dim', COUNT(*) FROM customer_dim
UNION ALL
SELECT '', 'account_dim', COUNT(*) FROM account_dim
UNION ALL
SELECT '', 'department_dim', COUNT(*) FROM department_dim
UNION ALL
SELECT '', 'region_dim', COUNT(*) FROM region_dim
UNION ALL
SELECT '', 'sales_rep_dim', COUNT(*) FROM sales_rep_dim
UNION ALL
SELECT '', 'campaign_dim', COUNT(*) FROM campaign_dim
UNION ALL
SELECT '', 'channel_dim', COUNT(*) FROM channel_dim
UNION ALL
SELECT '', 'employee_dim', COUNT(*) FROM employee_dim
UNION ALL
SELECT '', 'job_dim', COUNT(*) FROM job_dim
UNION ALL
SELECT '', 'location_dim', COUNT(*) FROM location_dim
UNION ALL
SELECT '', '', NULL
UNION ALL
SELECT 'FACT TABLES', '', NULL
UNION ALL
SELECT '', 'sales_fact', COUNT(*) FROM sales_fact
UNION ALL
SELECT '', 'finance_transactions', COUNT(*) FROM finance_transactions
UNION ALL
SELECT '', 'marketing_campaign_fact', COUNT(*) FROM marketing_campaign_fact
UNION ALL
SELECT '', 'hr_employee_fact', COUNT(*) FROM hr_employee_fact
UNION ALL
SELECT '', '', NULL
UNION ALL
SELECT 'SALESFORCE TABLES', '', NULL
UNION ALL
SELECT '', 'sf_accounts', COUNT(*) FROM sf_accounts
UNION ALL
SELECT '', 'sf_opportunities', COUNT(*) FROM sf_opportunities
UNION ALL
SELECT '', 'sf_contacts', COUNT(*) FROM sf_contacts;

# Part 2: Cortex Analyst Agent v1

This notebook focuses on creating the first version of our Cortex Analyst agent. This agent will have access to the core structured data via Cortex Analyst.

We will:
1. Create semantic views for our structured data.
2. Create the Cortex Analyst agent.
3. Test the agent with sample questions.

## 1. Semantic Views for Cortex Analyst

Create business unit-specific semantic views that enable natural language queries over our data warehouse.

Semantic Views are schema-level objects in Snowflake that enable us to define business metrics, entities, and their relationships. This context will be leveraged by Cortex Analyst for generating SQL to answer natural language questions.

Semantic Views are comprised of the following components:
- Tables: logical tables that map to Snowflake tables or views
- Facts: row-level attributes tied to a logical table that represent specific business events or transactions
- Dimensions: categorical attributes tied to a logical table that gives meaning to metrics by grouping data into meaningful categories
- Metrics: quantifiable measures of business performance calculated by aggregating facts or other columns from the same table
- Named filters: logic to filter a logical table based on some business rule
- Relationships: how logical tables are mapped to one another. This enables Cortex Analyst to join multiple logical tables together

### 1.1 Finance Semantic View

In [ ]:
CREATE OR REPLACE SEMANTIC VIEW SF_AI_DEMO.DEMO_SCHEMA.FINANCE_SEMANTIC_VIEW
    tables (
        TRANSACTIONS as FINANCE_TRANSACTIONS primary key (TRANSACTION_ID) with synonyms=('finance transactions','financial data') comment='All financial transactions across departments',
        ACCOUNTS as ACCOUNT_DIM primary key (ACCOUNT_KEY) with synonyms=('chart of accounts','account types') comment='Account dimension for financial categorization',
        DEPARTMENTS as DEPARTMENT_DIM primary key (DEPARTMENT_KEY) with synonyms=('business units','departments') comment='Department dimension for cost center analysis',
        VENDORS as VENDOR_DIM primary key (VENDOR_KEY) with synonyms=('suppliers','vendors') comment='Vendor information for spend analysis',
        PRODUCTS as PRODUCT_DIM primary key (PRODUCT_KEY) with synonyms=('products','items') comment='Product dimension for transaction analysis',
        CUSTOMERS as CUSTOMER_DIM primary key (CUSTOMER_KEY) with synonyms=('clients','customers') comment='Customer dimension for revenue analysis'
    )
    relationships (
        TRANSACTIONS_TO_ACCOUNTS as TRANSACTIONS(ACCOUNT_KEY) references ACCOUNTS(ACCOUNT_KEY),
        TRANSACTIONS_TO_DEPARTMENTS as TRANSACTIONS(DEPARTMENT_KEY) references DEPARTMENTS(DEPARTMENT_KEY),
        TRANSACTIONS_TO_VENDORS as TRANSACTIONS(VENDOR_KEY) references VENDORS(VENDOR_KEY),
        TRANSACTIONS_TO_PRODUCTS as TRANSACTIONS(PRODUCT_KEY) references PRODUCTS(PRODUCT_KEY),
        TRANSACTIONS_TO_CUSTOMERS as TRANSACTIONS(CUSTOMER_KEY) references CUSTOMERS(CUSTOMER_KEY)
    )
    facts (
        TRANSACTIONS.TRANSACTION_AMOUNT as amount comment='Transaction amount in dollars',
        TRANSACTIONS.TRANSACTION_RECORD as 1 comment='Count of transactions'
    )
    dimensions (
        TRANSACTIONS.TRANSACTION_DATE as date with synonyms=('date','transaction date') comment='Date of the financial transaction',
        TRANSACTIONS.TRANSACTION_MONTH as MONTH(date) comment='Month of the transaction',
        TRANSACTIONS.TRANSACTION_YEAR as YEAR(date) comment='Year of the transaction',
        ACCOUNTS.ACCOUNT_NAME as account_name with synonyms=('account','account type') comment='Name of the account',
        ACCOUNTS.ACCOUNT_TYPE as account_type with synonyms=('type','category') comment='Type of account (Income/Expense)',
        DEPARTMENTS.DEPARTMENT_NAME as department_name with synonyms=('department','business unit') comment='Name of the department',
        VENDORS.VENDOR_NAME as vendor_name with synonyms=('vendor','supplier') comment='Name of the vendor',
        PRODUCTS.PRODUCT_NAME as product_name with synonyms=('product','item') comment='Name of the product',
        CUSTOMERS.CUSTOMER_NAME as customer_name with synonyms=('customer','client') comment='Name of the customer'
    )
    metrics (
        TRANSACTIONS.AVERAGE_AMOUNT as AVG(transactions.amount) comment='Average transaction amount',
        TRANSACTIONS.TOTAL_AMOUNT as SUM(transactions.amount) comment='Total transaction amount',
        TRANSACTIONS.TOTAL_TRANSACTIONS as COUNT(transactions.transaction_record) comment='Total number of transactions'
    )
    comment='Semantic view for financial analysis and reporting';

### 1.2 Sales Semantic View

In [ ]:
CREATE OR REPLACE SEMANTIC VIEW SF_AI_DEMO.DEMO_SCHEMA.SALES_SEMANTIC_VIEW
	tables (
		CUSTOMERS as CUSTOMER_DIM primary key (CUSTOMER_KEY) with synonyms=('clients','customers','accounts') comment='Customer information for sales analysis',
		PRODUCTS as PRODUCT_DIM primary key (PRODUCT_KEY) with synonyms=('products','items','SKUs') comment='Product catalog for sales analysis',
		PRODUCT_CATEGORY_DIM primary key (CATEGORY_KEY),
		REGIONS as REGION_DIM primary key (REGION_KEY) with synonyms=('territories','regions','areas') comment='Regional information for territory analysis',
		SALES as SALES_FACT primary key (SALE_ID) with synonyms=('sales transactions','sales data') comment='All sales transactions and deals',
		SALES_REPS as SALES_REP_DIM primary key (SALES_REP_KEY) with synonyms=('sales representatives','reps','salespeople') comment='Sales representative information',
		VENDORS as VENDOR_DIM primary key (VENDOR_KEY) with synonyms=('suppliers','vendors') comment='Vendor information for supply chain analysis'
	)
	relationships (
		PRODUCT_TO_CATEGORY as PRODUCTS(CATEGORY_KEY) references PRODUCT_CATEGORY_DIM(CATEGORY_KEY),
		SALES_TO_CUSTOMERS as SALES(CUSTOMER_KEY) references CUSTOMERS(CUSTOMER_KEY),
		SALES_TO_PRODUCTS as SALES(PRODUCT_KEY) references PRODUCTS(PRODUCT_KEY),
		SALES_TO_REGIONS as SALES(REGION_KEY) references REGIONS(REGION_KEY),
		SALES_TO_REPS as SALES(SALES_REP_KEY) references SALES_REPS(SALES_REP_KEY),
		SALES_TO_VENDORS as SALES(VENDOR_KEY) references VENDORS(VENDOR_KEY)
	)
	facts (
		SALES.SALE_AMOUNT as amount comment='Sale amount in dollars',
		SALES.SALE_RECORD as 1 comment='Count of sales transactions',
		SALES.UNITS_SOLD as units comment='Number of units sold'
	)
	dimensions (
		CUSTOMERS.CUSTOMER_INDUSTRY as INDUSTRY with synonyms=('industry','customer type') comment='Customer industry',
		CUSTOMERS.CUSTOMER_NAME as customer_name with synonyms=('customer','client','account') comment='Name of the customer',
		PRODUCTS.PRODUCT_NAME as product_name with synonyms=('product','item') comment='Name of the product',
		PRODUCT_CATEGORY_DIM.CATEGORY_NAME as CATEGORY_NAME with synonyms=('category','product category') comment='The category to which a product belongs',
		PRODUCT_CATEGORY_DIM.VERTICAL as VERTICAL with synonyms=('industry','sector','market') comment='The industry or sector in which a product is categorized',
		REGIONS.REGION_NAME as region_name with synonyms=('region','territory','area') comment='Name of the region',
		SALES.SALE_DATE as date with synonyms=('date','sale date','transaction date') comment='Date of the sale',
		SALES.SALE_MONTH as MONTH(date) comment='Month of the sale',
		SALES.SALE_YEAR as YEAR(date) comment='Year of the sale',
		SALES_REPS.SALES_REP_NAME as REP_NAME with synonyms=('sales rep','representative','salesperson') comment='Name of the sales representative',
		VENDORS.VENDOR_NAME as vendor_name with synonyms=('vendor','supplier','provider') comment='Name of the vendor'
	)
	metrics (
		SALES.AVERAGE_DEAL_SIZE as AVG(sales.amount) comment='Average deal size',
		SALES.AVERAGE_UNITS_PER_SALE as AVG(sales.units) comment='Average units per sale',
		SALES.TOTAL_DEALS as COUNT(sales.sale_record) comment='Total number of deals',
		SALES.TOTAL_REVENUE as SUM(sales.amount) comment='Total sales revenue',
		SALES.TOTAL_UNITS as SUM(sales.units) comment='Total units sold'
	)
	comment='Semantic view for sales analysis and performance tracking';

### 1.3 Marketing Semantic View

In [ ]:
CREATE OR REPLACE SEMANTIC VIEW SF_AI_DEMO.DEMO_SCHEMA.MARKETING_SEMANTIC_VIEW
	tables (
		ACCOUNTS as SF_ACCOUNTS primary key (ACCOUNT_ID) with synonyms=('customers','accounts','clients') comment='Customer account information for revenue analysis',
		CAMPAIGNS as MARKETING_CAMPAIGN_FACT primary key (CAMPAIGN_FACT_ID) with synonyms=('marketing campaigns','campaign data') comment='Marketing campaign performance data',
		CAMPAIGN_DETAILS as CAMPAIGN_DIM primary key (CAMPAIGN_KEY) with synonyms=('campaign info','campaign details') comment='Campaign dimension with objectives and names',
		CHANNELS as CHANNEL_DIM primary key (CHANNEL_KEY) with synonyms=('marketing channels','channels') comment='Marketing channel information',
		CONTACTS as SF_CONTACTS primary key (CONTACT_ID) with synonyms=('leads','contacts','prospects') comment='Contact records generated from marketing campaigns',
		OPPORTUNITIES as SF_OPPORTUNITIES primary key (OPPORTUNITY_ID) with synonyms=('deals','opportunities','sales pipeline') comment='Sales opportunities and revenue data',
		PRODUCTS as PRODUCT_DIM primary key (PRODUCT_KEY) with synonyms=('products','items') comment='Product dimension for campaign-specific analysis',
		REGIONS as REGION_DIM primary key (REGION_KEY) with synonyms=('territories','regions','markets') comment='Regional information for campaign analysis'
	)
	relationships (
		CAMPAIGNS_TO_CHANNELS as CAMPAIGNS(CHANNEL_KEY) references CHANNELS(CHANNEL_KEY),
		CAMPAIGNS_TO_DETAILS as CAMPAIGNS(CAMPAIGN_KEY) references CAMPAIGN_DETAILS(CAMPAIGN_KEY),
		CAMPAIGNS_TO_PRODUCTS as CAMPAIGNS(PRODUCT_KEY) references PRODUCTS(PRODUCT_KEY),
		CAMPAIGNS_TO_REGIONS as CAMPAIGNS(REGION_KEY) references REGIONS(REGION_KEY),
		CONTACTS_TO_ACCOUNTS as CONTACTS(ACCOUNT_ID) references ACCOUNTS(ACCOUNT_ID),
		CONTACTS_TO_CAMPAIGNS as CONTACTS(CAMPAIGN_NO) references CAMPAIGNS(CAMPAIGN_FACT_ID),
		OPPORTUNITIES_TO_ACCOUNTS as OPPORTUNITIES(ACCOUNT_ID) references ACCOUNTS(ACCOUNT_ID),
		OPPORTUNITIES_TO_CAMPAIGNS as OPPORTUNITIES(CAMPAIGN_ID) references CAMPAIGNS(CAMPAIGN_FACT_ID)
	)
	facts (
		CAMPAIGNS.CAMPAIGN_RECORD as 1 comment='Count of campaign activities',
		CAMPAIGNS.CAMPAIGN_SPEND as spend comment='Marketing spend in dollars',
		CAMPAIGNS.IMPRESSIONS as IMPRESSIONS comment='Number of impressions',
		CAMPAIGNS.LEADS_GENERATED as LEADS_GENERATED comment='Number of leads generated',
		CONTACTS.CONTACT_RECORD as 1 comment='Count of contacts generated',
		OPPORTUNITIES.OPPORTUNITY_RECORD as 1 comment='Count of opportunities created',
		OPPORTUNITIES.REVENUE as AMOUNT comment='Opportunity revenue in dollars'
	)
	dimensions (
		ACCOUNTS.ACCOUNT_NAME as ACCOUNT_NAME with synonyms=('customer name','client name','company') comment='Name of the customer account',
		ACCOUNTS.INDUSTRY as INDUSTRY with synonyms=('industry','sector') comment='Customer industry',
		CAMPAIGNS.CAMPAIGN_DATE as date with synonyms=('date','campaign date') comment='Date of the campaign activity',
		CAMPAIGNS.CAMPAIGN_MONTH as MONTH(date) comment='Month of the campaign',
		CAMPAIGNS.CAMPAIGN_YEAR as YEAR(date) comment='Year of the campaign',
		CAMPAIGN_DETAILS.CAMPAIGN_NAME as CAMPAIGN_NAME with synonyms=('campaign','campaign title') comment='Name of the marketing campaign',
		CAMPAIGN_DETAILS.CAMPAIGN_OBJECTIVE as OBJECTIVE with synonyms=('objective','goal','purpose') comment='Campaign objective',
		CHANNELS.CHANNEL_NAME as CHANNEL_NAME with synonyms=('channel','marketing channel') comment='Name of the marketing channel',
		OPPORTUNITIES.OPPORTUNITY_STAGE as STAGE_NAME comment='Stage name of the opportunity. Closed Won indicates an actual sale with revenue',
		OPPORTUNITIES.OPPORTUNITY_NAME as OPPORTUNITY_NAME with synonyms=('deal name','opportunity title') comment='Name of the sales opportunity',
		PRODUCTS.PRODUCT_NAME as PRODUCT_NAME with synonyms=('product','item','product title') comment='Name of the product being promoted',
		REGIONS.REGION_NAME as REGION_NAME with synonyms=('region','market','territory') comment='Name of the region'
	)
	metrics (
		CAMPAIGNS.TOTAL_SPEND as SUM(CAMPAIGNS.spend) comment='Total marketing spend',
		CAMPAIGNS.TOTAL_IMPRESSIONS as SUM(CAMPAIGNS.impressions) comment='Total impressions across campaigns',
		CAMPAIGNS.TOTAL_LEADS as SUM(CAMPAIGNS.leads_generated) comment='Total leads generated from campaigns',
		OPPORTUNITIES.TOTAL_OPPORTUNITIES as COUNT(OPPORTUNITIES.opportunity_record) comment='Total opportunities from marketing',
		OPPORTUNITIES.CLOSED_WON_REVENUE as SUM(CASE WHEN OPPORTUNITIES.opportunity_stage = 'Closed Won' THEN OPPORTUNITIES.revenue ELSE 0 END) comment='Revenue from closed won opportunities',
		OPPORTUNITIES.TOTAL_REVENUE as SUM(OPPORTUNITIES.revenue) comment='Total revenue from marketing-driven opportunities'
	)
	comment='Semantic view for marketing campaign analysis with complete revenue attribution and ROI tracking';

### 1.4 HR Semantic View

In [ ]:
CREATE OR REPLACE SEMANTIC VIEW SF_AI_DEMO.DEMO_SCHEMA.HR_SEMANTIC_VIEW
	tables (
		DEPARTMENTS as DEPARTMENT_DIM primary key (DEPARTMENT_KEY) with synonyms=('departments','business units') comment='Department dimension for organizational analysis',
		EMPLOYEES as EMPLOYEE_DIM primary key (EMPLOYEE_KEY) with synonyms=('employees','staff','workforce') comment='Employee dimension with personal information',
		HR_RECORDS as HR_EMPLOYEE_FACT primary key (HR_FACT_ID) with synonyms=('hr data','employee records') comment='HR employee fact data for workforce analysis',
		JOBS as JOB_DIM primary key (JOB_KEY) with synonyms=('job titles','positions','roles') comment='Job dimension with titles and levels',
		LOCATIONS as LOCATION_DIM primary key (LOCATION_KEY) with synonyms=('locations','offices','sites') comment='Location dimension for geographic analysis'
	)
	relationships (
		HR_TO_DEPARTMENTS as HR_RECORDS(DEPARTMENT_KEY) references DEPARTMENTS(DEPARTMENT_KEY),
		HR_TO_EMPLOYEES as HR_RECORDS(EMPLOYEE_KEY) references EMPLOYEES(EMPLOYEE_KEY),
		HR_TO_JOBS as HR_RECORDS(JOB_KEY) references JOBS(JOB_KEY),
		HR_TO_LOCATIONS as HR_RECORDS(LOCATION_KEY) references LOCATIONS(LOCATION_KEY)
	)
	facts (
		HR_RECORDS.ATTRITION_FLAG as attrition_flag with synonyms=('turnover_indicator','employee_departure_flag','churn_status') comment='Attrition flag. value is 0 if employee is currently active. 1 if employee quit & left the company. Always filter by 0 to show active employees unless specified otherwise',
		HR_RECORDS.EMPLOYEE_RECORD as 1 comment='Count of employee records',
		HR_RECORDS.EMPLOYEE_SALARY as salary comment='Employee salary in dollars'
	)
	dimensions (
		DEPARTMENTS.DEPARTMENT_NAME as department_name with synonyms=('department','business unit','division') comment='Name of the department',
		EMPLOYEES.EMPLOYEE_NAME as employee_name with synonyms=('employee','staff member','person') comment='Name of the employee',
		EMPLOYEES.GENDER as gender with synonyms=('gender','sex') comment='Employee gender',
		EMPLOYEES.HIRE_DATE as hire_date with synonyms=('hire date','start date') comment='Date when employee was hired',
		HR_RECORDS.RECORD_DATE as date with synonyms=('date','record date') comment='Date of the HR record',
		HR_RECORDS.RECORD_MONTH as MONTH(date) comment='Month of the HR record',
		HR_RECORDS.RECORD_YEAR as YEAR(date) comment='Year of the HR record',
		JOBS.JOB_TITLE as job_title with synonyms=('job title','position','role') comment='Employee job title',
		JOBS.JOB_LEVEL as job_level with synonyms=('level','grade','seniority') comment='Job level or grade',
		LOCATIONS.LOCATION_NAME as location_name with synonyms=('location','office','site') comment='Work location'
	)
	metrics (
		HR_RECORDS.TOTAL_EMPLOYEES as COUNT(hr_records.employee_record) comment='Total number of employees',
		HR_RECORDS.ATTRITION_COUNT as SUM(hr_records.attrition_flag) comment='Number of employees who left',
		HR_RECORDS.AVG_SALARY as AVG(hr_records.employee_salary) comment='Average employee salary',
		HR_RECORDS.TOTAL_SALARY_COST as SUM(hr_records.employee_salary) comment='Total salary cost'
	)
	comment='Semantic view for HR analytics and workforce management';

### 1.5 Verify Semantic Views

In [ ]:
-- Show all semantic views
SHOW SEMANTIC VIEWS;

## 2. Create and Test v1 Cortex Agent

Cortex Agents are the brains of this demo. Agents will orchestrate both structured and unstructured data to deliver insights. They plan tasks, use tools to execute these tasks, and generate responses. Agents can be configured to use Cortex Analyst (structured data), Cortex Search (unstructured data), and custom tools to generate answers.

Below we create the first version of our Agent, using the various Semantic Views created above to power Cortex Analyst.

In [ ]:
-- Create the first version of our Company Chatbot Agent
USE ROLE SF_Intelligence_Demo;

CREATE OR REPLACE AGENT SNOWFLAKE_INTELLIGENCE.AGENTS.Company_Chatbot_Agent_Structured
WITH PROFILE='{"display_name": "1-Company Chatbot Agent - Structured Data"}'
COMMENT=$$ This is an agent that can answer questions about company specific Sales, Marketing, HR & Finance questions. $$
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": ""
  },
  "instructions": {
    "response": "You are a data analyst who has access to sales, finance, marketing & HR datamarts. If user does not specify a date range assume it for year 2025. Leverage data from all domains to analyse & answer user questions. Provide visualizations if possible. Trendlines should default to linecharts, Categories Barchart.",
    "orchestration": "Use cortex search for known entities and pass the results to cortex analyst for detailed analysis.\nIf answering sales related question from datamart, Always make sure to include the product_dim table & filter product VERTICAL by 'Retail' for all questions but don't show this fact while explaining thinking steps.\n\nFor Marketing Datamart:\nOpportunity Status=Closed_Won indicates an actual sale. \nSalesID in marketing datamart links an opportunity to a Sales record in Sales Datamart SalesID columns\n\n\n",
    "sample_questions": [
      {
        "question": "What are our monthly sales last 12 months?"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Finance Datamart",
        "description": "Allows users to query finance data for a company in terms of revenue & expenses."
      }
    },
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Sales Datamart",
        "description": "Allows users to query Sales data for a company in terms of Sales data such as products, sales reps & etc. "
      }
    },
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Marketing Datamart",
        "description": "Allows users to query Marketing data in terms of campaigns, channels, impressions, spend & etc."
      }
    },
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query HR Datamart",
        "description": "Allows users to query HR data for a company in terms of HR related employee data. employee_name column also contains names of sales_reps."
      }
    }
  ],
  "tool_resources": {
    "Query Finance Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.FINANCE_SEMANTIC_VIEW"
    },
    "Query Sales Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.SALES_SEMANTIC_VIEW"
    },
    "Query Marketing Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.MARKETING_SEMANTIC_VIEW"
    },
    "Query HR Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.HR_SEMANTIC_VIEW"
    }
  }
}
$$;

### 2.1 Test Analyst Agent v1!

Open Snowflake Intelligence by clicking on 'AI & ML' on the left navbar and then choosing 'Snowflake Intelligence'.

Once there, ask away about our structured data! Here are some sample questions as thought starters:

#### Sales
- What was our total sales revenue in 2025?
- What are our top 5 products by revenue in 2025? Show me their performance by region.
- Show me monthly sales trends for 2025 with visualizations. Which months had the highest revenue?

#### HR
- Show me employee headcount and average salary by department. Which departments have the highest attrition rates?

#### Marketing
- Which marketing campaigns generated the most revenue in 2025? Show me marketing ROI and cost per lead by channel.
- Compare marketing spend to actual closed revenue by channel. Which channels drive the highest value customers?

# Part 3: Cortex Search and Custom Tools

This section enhances our Cortex Analyst agent with unstructured data search capabilities and custom tools.

Cortex Search is how we enable Cortex Agents to derive insights from unstructured data. Cortex Search will take chunks of text data, embed them, and store them in a performant index for fast retrieval and similarity search. Cortex Search also enables automatic refresh of the index as new unstructured data becomes available.

We will:
1. Create a Cortex Search service for our unstructured documents.
2. Create custom Python functions for specialized logic.
3. Update the agent to use these new tools.
4. Test the enhanced agent with new types of questions.



## 1. Unstructured Document Processing
Process PDF documents from the internal stage and prepare them for search capabilities. This creates a foundation for document-based AI interactions.

In [ ]:
-- Create table to store parsed document content
CREATE OR REPLACE TABLE parsed_content AS 
SELECT 
    relative_path, 
    BUILD_STAGE_FILE_URL('@SF_AI_DEMO.DEMO_SCHEMA.INTERNAL_DATA_STAGE', relative_path) as file_url,
    TO_FILE(BUILD_STAGE_FILE_URL('@SF_AI_DEMO.DEMO_SCHEMA.INTERNAL_DATA_STAGE', relative_path)) file_object,
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
        @SF_AI_DEMO.DEMO_SCHEMA.INTERNAL_DATA_STAGE,
        relative_path,
        {'mode':'LAYOUT'}
    ):content::string as Content
FROM directory(@SF_AI_DEMO.DEMO_SCHEMA.INTERNAL_DATA_STAGE) 
WHERE relative_path ILIKE 'unstructured_docs/%.pdf';

## 2. Cortex Search Services

Create department-specific search services that enable semantic search over unstructured documents. These services use embeddings to understand document content and enable natural language document search.

In [ ]:
-- Create search service for finance documents
CREATE OR REPLACE CORTEX SEARCH SERVICE Search_finance_docs
    ON content
    ATTRIBUTES relative_path, file_url, title
    WAREHOUSE = SNOW_INTELLIGENCE_DEMO_WH
    TARGET_LAG = '30 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
        SELECT
            relative_path,
            file_url,
            REGEXP_SUBSTR(relative_path, '[^/]+$') as title, -- Extract filename as title
            content
        FROM parsed_content
        WHERE relative_path ILIKE '%/finance/%'
    );

-- Create search service for HR documents
CREATE OR REPLACE CORTEX SEARCH SERVICE Search_hr_docs
    ON content
    ATTRIBUTES relative_path, file_url, title
    WAREHOUSE = SNOW_INTELLIGENCE_DEMO_WH
    TARGET_LAG = '30 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
        SELECT
            relative_path,
            file_url,
            REGEXP_SUBSTR(relative_path, '[^/]+$') as title,
            content
        FROM parsed_content
        WHERE relative_path ILIKE '%/hr/%'
    );

-- Create search service for marketing documents
CREATE OR REPLACE CORTEX SEARCH SERVICE Search_marketing_docs
    ON content
    ATTRIBUTES relative_path, file_url, title
    WAREHOUSE = SNOW_INTELLIGENCE_DEMO_WH
    TARGET_LAG = '30 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
        SELECT
            relative_path,
            file_url,
            REGEXP_SUBSTR(relative_path, '[^/]+$') as title,
            content
        FROM parsed_content
        WHERE relative_path ILIKE '%/marketing/%'
    );

-- Create search service for sales documents
CREATE OR REPLACE CORTEX SEARCH SERVICE Search_sales_docs
    ON content
    ATTRIBUTES relative_path, file_url, title
    WAREHOUSE = SNOW_INTELLIGENCE_DEMO_WH
    TARGET_LAG = '30 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
        SELECT
            relative_path,
            file_url,
            REGEXP_SUBSTR(relative_path, '[^/]+$') as title,
            content
        FROM parsed_content
        WHERE relative_path ILIKE '%/sales/%'
    );

## 3. Update the Agent
Update the agent to include the newly created Cortex Search Services which will enable questions against the unstructured data.

In [ ]:
-- Create the comprehensive Company Chatbot Agent
CREATE OR REPLACE AGENT SNOWFLAKE_INTELLIGENCE.AGENTS.Company_Chatbot_Agent_Unstructured
WITH PROFILE='{"display_name": "2-Company Chatbot Agent - Unstructured"}'
COMMENT=$$ This is an agent that can answer questions about company specific Sales, Marketing, HR & Finance questions. $$
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": ""
  },
  "instructions": {
    "response": "You are a data analyst who has access to sales, finance, marketing & HR datamarts. If user does not specify a date range assume it for year 2025. Leverage data from all domains to analyse & answer user questions. Provide visualizations if possible. Trendlines should default to linecharts, Categories Barchart.",
    "orchestration": "Use cortex search for known entities and pass the results to cortex analyst for detailed analysis.\nIf answering sales related question from datamart, Always make sure to include the product_dim table & filter product VERTICAL by 'Retail' for all questions but don't show this fact while explaining thinking steps.\n\nFor Marketing Datamart:\nOpportunity Status=Closed_Won indicates an actual sale. \nSalesID in marketing datamart links an opportunity to a Sales record in Sales Datamart SalesID columns\n\n\n",
    "sample_questions": [
      {
        "question": "What are our monthly sales last 12 months?"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Finance Datamart",
        "description": "Allows users to query finance data for a company in terms of revenue & expenses."
      }
    },
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Sales Datamart",
        "description": "Allows users to query Sales data for a company in terms of Sales data such as products, sales reps & etc. "
      }
    },
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query HR Datamart",
        "description": "Allows users to query HR data for a company in terms of HR related employee data. employee_name column also contains names of sales_reps."
      }
    },
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Marketing Datamart",
        "description": "Allows users to query Marketing data in terms of campaigns, channels, impressions, spend & etc."
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "Search Internal Documents: Finance",
        "description": ""
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "Search Internal Documents: HR",
        "description": ""
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "Search Internal Documents: Sales",
        "description": ""
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "Search Internal Documents: Marketing",
        "description": "This tools should be used to search unstructured docs related to marketing department.\n\nAny reference docs in ID columns should be passed to Dynamic URL tool to generate a downloadable URL for users in the response"
      }
    },
  ],
  "tool_resources": {
    "Query Finance Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.FINANCE_SEMANTIC_VIEW"
    },
    "Query HR Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.HR_SEMANTIC_VIEW"
    },
    "Query Marketing Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.MARKETING_SEMANTIC_VIEW"
    },
    "Query Sales Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.SALES_SEMANTIC_VIEW"
    },
    "Search Internal Documents: Finance": {
      "id_column": "FILE_URL",
      "max_results": 5,
      "name": "SF_AI_DEMO.DEMO_SCHEMA.SEARCH_FINANCE_DOCS",
      "title_column": "TITLE"
    },
    "Search Internal Documents: HR": {
      "id_column": "FILE_URL",
      "max_results": 5,
      "name": "SF_AI_DEMO.DEMO_SCHEMA.SEARCH_HR_DOCS",
      "title_column": "TITLE"
    },
    "Search Internal Documents: Marketing": {
      "id_column": "RELATIVE_PATH",
      "max_results": 5,
      "name": "SF_AI_DEMO.DEMO_SCHEMA.SEARCH_MARKETING_DOCS",
      "title_column": "TITLE"
    },
    "Search Internal Documents: Sales": {
      "id_column": "FILE_URL",
      "max_results": 5,
      "name": "SF_AI_DEMO.DEMO_SCHEMA.SEARCH_SALES_DOCS",
      "title_column": "TITLE"
    }
  }
}
$$;

## 4. Ask questions about the unstructured data
Go back to Snowflake Intelligence and refresh the page to see our new agent added with the unstructured data. Try asking questions about the unstructured data, such as:
- What is the company policy on expense reports?
- What are our top 5 vendors in the last 5 years? Check our vendor management policy - are we following procurement guidelines for all transactions?

## 5. Custom Tools / Logic

Create supporting stored procedures and functions that will be used by the Snowflake Intelligence Agent for file access, email sending, and web scraping capabilities.

In [ ]:
-- Create stored procedure to generate presigned URLs for files in internal stages
-- This will enable users to download files referenced from answers in Snowflake Intelligence
CREATE OR REPLACE PROCEDURE Get_File_Presigned_URL_SP(
    RELATIVE_FILE_PATH STRING, 
    EXPIRATION_MINS INTEGER DEFAULT 60
)
RETURNS STRING
LANGUAGE SQL
COMMENT = 'Generates a presigned URL for a file in the static @INTERNAL_DATA_STAGE. Input is the relative file path.'
EXECUTE AS CALLER
AS
$$
DECLARE
    presigned_url STRING;
    sql_stmt STRING;
    expiration_seconds INTEGER;
    stage_name STRING DEFAULT '@SF_AI_DEMO.DEMO_SCHEMA.INTERNAL_DATA_STAGE';
BEGIN
    expiration_seconds := EXPIRATION_MINS * 60;

    sql_stmt := 'SELECT GET_PRESIGNED_URL(' || stage_name || ', ' || '''' || RELATIVE_FILE_PATH || '''' || ', ' || expiration_seconds || ') AS url';
    
    EXECUTE IMMEDIATE :sql_stmt;
    
    SELECT "URL"
    INTO :presigned_url
    FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()));
    
    RETURN :presigned_url;
END;
$$;

In [ ]:
-- Create stored procedure to send emails to verified recipients in Snowflake
CREATE OR REPLACE PROCEDURE send_mail(recipient TEXT, subject TEXT, text TEXT)
RETURNS TEXT
LANGUAGE PYTHON
RUNTIME_VERSION = '3.11'
PACKAGES = ('snowflake-snowpark-python')
HANDLER = 'send_mail'
AS
$$
def send_mail(session, recipient, subject, text):
    session.call(
        'SYSTEM$SEND_EMAIL',
        'ai_email_int',
        recipient,
        subject,
        text,
        'text/html'
    )
    return f'Email was sent to {recipient} with subject: "{subject}".'
$$;

In [ ]:
-- Create web scraping function for external content analysis
CREATE OR REPLACE FUNCTION Web_scrape(weburl STRING)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.11
HANDLER = 'get_page'
EXTERNAL_ACCESS_INTEGRATIONS = (Snowflake_intelligence_ExternalAccess_Integration)
PACKAGES = ('requests', 'beautifulsoup4')
AS
$$
import _snowflake
import requests
from bs4 import BeautifulSoup

def get_page(weburl):
  url = f"{weburl}"
  response = requests.get(url)
  soup = BeautifulSoup(response.text)
  return soup.get_text()
$$;

## 6. Update Agent with Search and Custom Tools

In [ ]:
-- Create the comprehensive Company Chatbot Agent
CREATE OR REPLACE AGENT SNOWFLAKE_INTELLIGENCE.AGENTS.Company_Chatbot_Agent_Full
WITH PROFILE='{"display_name": "3-Company Chatbot Agent - Full"}'
COMMENT=$$ This is an agent that can answer questions about company specific Sales, Marketing, HR & Finance questions. $$
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": ""
  },
  "instructions": {
    "response": "You are a data analyst who has access to sales, finance, marketing & HR datamarts. If user does not specify a date range assume it for year 2025. Leverage data from all domains to analyse & answer user questions. Provide visualizations if possible. Trendlines should default to linecharts, Categories Barchart.",
    "orchestration": "Use cortex search for known entities and pass the results to cortex analyst for detailed analysis.\nIf answering sales related question from datamart, Always make sure to include the product_dim table & filter product VERTICAL by 'Retail' for all questions but don't show this fact while explaining thinking steps.\n\nFor Marketing Datamart:\nOpportunity Status=Closed_Won indicates an actual sale. \nSalesID in marketing datamart links an opportunity to a Sales record in Sales Datamart SalesID columns\n\n\n",
    "sample_questions": [
      {
        "question": "What are our monthly sales last 12 months?"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Finance Datamart",
        "description": "Allows users to query finance data for a company in terms of revenue & expenses."
      }
    },
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Sales Datamart",
        "description": "Allows users to query Sales data for a company in terms of Sales data such as products, sales reps & etc. "
      }
    },
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query HR Datamart",
        "description": "Allows users to query HR data for a company in terms of HR related employee data. employee_name column also contains names of sales_reps."
      }
    },
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Marketing Datamart",
        "description": "Allows users to query Marketing data in terms of campaigns, channels, impressions, spend & etc."
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "Search Internal Documents: Finance",
        "description": ""
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "Search Internal Documents: HR",
        "description": ""
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "Search Internal Documents: Sales",
        "description": ""
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "Search Internal Documents: Marketing",
        "description": "This tools should be used to search unstructured docs related to marketing department.\n\nAny reference docs in ID columns should be passed to Dynamic URL tool to generate a downloadable URL for users in the response"
      }
    },
    {
      "tool_spec": {
        "type": "generic",
        "name": "Web_scraper",
        "description": "This tool should be used if the user wants to analyse contents of a given web page. This tool will use a web url (https or https) as input and will return the text content of that web page for further analysis",
        "input_schema": {
          "type": "object",
          "properties": {
            "weburl": {
              "description": "Agent should ask web url ( that includes http:// or https:// ). It will scrape text from the given url and return as a result.",
              "type": "string"
            }
          },
          "required": [
            "weburl"
          ]
        }
      }
    },
    {
      "tool_spec": {
        "type": "generic",
        "name": "Send_Emails",
        "description": "This tool is used to send emails to a email recipient. It can take an email, subject & content as input to send the email. Always use HTML formatted content for the emails.",
        "input_schema": {
          "type": "object",
          "properties": {
            "recipient": {
              "description": "recipient of email",
              "type": "string"
            },
            "subject": {
              "description": "subject of email",
              "type": "string"
            },
            "text": {
              "description": "content of email",
              "type": "string"
            }
          },
          "required": [
            "text",
            "recipient",
            "subject"
          ]
        }
      }
    },
    {
      "tool_spec": {
        "type": "generic",
        "name": "Dynamic_Doc_URL_Tool",
        "description": "This tools uses the ID Column coming from Cortex Search tools for reference docs and returns a temp URL for users to view & download the docs.\n\nReturned URL should be presented as a HTML Hyperlink where doc title should be the text and out of this tool should be the url.\n\nURL format for PDF docs that are are like this which has no PDF in the url. Create the Hyperlink format so the PDF doc opens up in a browser instead of downloading the file.\nhttps://domain/path/unique_guid",
        "input_schema": {
          "type": "object",
          "properties": {
            "expiration_mins": {
              "description": "default should be 5",
              "type": "number"
            },
            "relative_file_path": {
              "description": "This is the ID Column value Coming from Cortex Search tool.",
              "type": "string"
            }
          },
          "required": [
            "expiration_mins",
            "relative_file_path"
          ]
        }
      }
    }
  ],
  "tool_resources": {
    "Dynamic_Doc_URL_Tool": {
      "execution_environment": {
        "query_timeout": 0,
        "type": "warehouse",
        "warehouse": "SNOW_INTELLIGENCE_DEMO_WH"
      },
      "identifier": "SF_AI_DEMO.DEMO_SCHEMA.GET_FILE_PRESIGNED_URL_SP",
      "name": "GET_FILE_PRESIGNED_URL_SP(VARCHAR, DEFAULT NUMBER)",
      "type": "procedure"
    },
    "Query Finance Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.FINANCE_SEMANTIC_VIEW"
    },
    "Query HR Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.HR_SEMANTIC_VIEW"
    },
    "Query Marketing Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.MARKETING_SEMANTIC_VIEW"
    },
    "Query Sales Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.SALES_SEMANTIC_VIEW"
    },
    "Search Internal Documents: Finance": {
      "id_column": "FILE_URL",
      "max_results": 5,
      "name": "SF_AI_DEMO.DEMO_SCHEMA.SEARCH_FINANCE_DOCS",
      "title_column": "TITLE"
    },
    "Search Internal Documents: HR": {
      "id_column": "FILE_URL",
      "max_results": 5,
      "name": "SF_AI_DEMO.DEMO_SCHEMA.SEARCH_HR_DOCS",
      "title_column": "TITLE"
    },
    "Search Internal Documents: Marketing": {
      "id_column": "RELATIVE_PATH",
      "max_results": 5,
      "name": "SF_AI_DEMO.DEMO_SCHEMA.SEARCH_MARKETING_DOCS",
      "title_column": "TITLE"
    },
    "Search Internal Documents: Sales": {
      "id_column": "FILE_URL",
      "max_results": 5,
      "name": "SF_AI_DEMO.DEMO_SCHEMA.SEARCH_SALES_DOCS",
      "title_column": "TITLE"
    },
    "Send_Emails": {
      "execution_environment": {
        "query_timeout": 0,
        "type": "warehouse",
        "warehouse": "SNOW_INTELLIGENCE_DEMO_WH"
      },
      "identifier": "SF_AI_DEMO.DEMO_SCHEMA.SEND_MAIL",
      "name": "SEND_MAIL(VARCHAR, VARCHAR, VARCHAR)",
      "type": "procedure"
    }
  }
}
$$;

-- ,
--     "Web_scraper": {
--       "execution_environment": {
--         "query_timeout": 0,
--         "type": "warehouse",
--         "warehouse": "SNOW_INTELLIGENCE_DEMO_WH"
--       },
--       "identifier": "SF_AI_DEMO.DEMO_SCHEMA.WEB_SCRAPE",
--       "name": "WEB_SCRAPE(VARCHAR)",
--       "type": "function"
--     }

## 7. Test Full Agent

Below is a full suite of questions that are possible with our final agent:

🎯 Sales Performance Analysis
- "Show me monthly sales trends for 2025 with visualizations. Which months had the highest revenue?"
- "What are our top 5 products by revenue in 2025? Show me their performance by region."
- "Who are our top performing sales representatives? Show their individual revenue contributions and deal counts."

👥 HR & Workforce Analysis
- "What is the average tenure of our top sales reps? Is there a correlation between tenure and sales performance?"
- "Show me employee headcount and average salary by department. Which departments have the highest attrition rates?"
- "How are our employees distributed across locations? What are the performance differences by location?"

📈 Marketing Campaign Effectiveness & Revenue Attribution
- "Which marketing campaigns generated the most revenue in 2025? Show me marketing ROI and cost per lead by channel."
- "Show me the complete marketing funnel from impressions to closed revenue. Which campaigns have the best conversion rates?"
- "Compare marketing spend to actual closed revenue by channel. Which channels drive the highest value customers?"

💰 Finance & Cross-Domain Integration
- "Show me revenue generated by each marketing channel. What is our true marketing ROI from campaigns to closed deals?"
- "What are our top 5 vendors in the last 5 years? Check our vendor management policy - are we following procurement guidelines for all transactions?"

🔍 Cross-Functional Insights & External Data
Web Content Analysis Questions
- "Analyze the content from [competitor website URL] and compare their product offerings to our product catalog."
- "Scrape content from [industry report URL] and analyze how it relates to our sales performance and market positioning."
- "Get the latest information from [company news URL] and analyze its potential impact on our sales forecast."